In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

# import argparse

/home/yarden/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/yarden/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [11]:
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool  # , Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, show  # , output_notebook

In [12]:
# output_notebook()

Loading BokehJS ...

In [4]:
# import stock data from CSV
stocks_data_fname_lastest = '../assets/stocks_raw_data_latest.csv'

# stocks_data.to_csv(stocks_data_fname_lastest, 
#                    sep=';', na_rep='NaN', index=False, encoding='UTF-8')

stocks_data = pd.read_csv(
    stocks_data_fname_lastest, 
    sep=';', 
    na_values='NaN', 
    header=0,
    index_col=0,
    encoding='UTF-8')

In [5]:
stocks_data['color'] = np.where(stocks_data['% תשואה 3 חודשים אחרונים']\
                                > 0, 'orange', 'grey')
stocks_data['alpha'] = np.where(stocks_data['% תשואה 3 חודשים אחרונים']\
                                > 0, 0.9, 0.25)

In [6]:
# stocks_data.columns

In [7]:
# stocks_data.head(5)

In [13]:
# Create Input controls
cast = TextInput(title="Cast names contains")
x_axis = Select(title="X Axis", options=sorted(list(stocks_data.columns)), 
                value=sorted(list(stocks_data.columns))[0])
y_axis = Select(title="Y Axis", options=sorted(list(stocks_data.columns)), 
                value=sorted(list(stocks_data.columns))[1])

# Create Column Data Source that will be used by the plot
source = ColumnDataSource(data=dict(x=[], y=[], 
                                    color=[], 
                                    title=[], 
                                    # year=[], revenue=[], 
                                    alpha=[]))

hover = HoverTool(tooltips=[
    ('Name', '@stock_name'),
    ('Market Cap (thousands)', '@stock_market_cap'),
    ('Beta', '@stock_beta'),
#     ("$", "@revenue")
])

p = figure(plot_height=600, plot_width=700, title='', toolbar_location=None, tools=[hover])
p.circle(x='x', y='y',
         source=source, size=7, color='color', line_color=None, fill_alpha='alpha')


def select_movies():
    cast_val = cast.value.strip()
    selected = stocks_data  # [
#         (movies.Reviews >= reviews.value) &
#         (movies.BoxOffice >= (boxoffice.value * 1e6)) &
#         (movies.Year >= min_year.value) &
#         (movies.Year <= max_year.value) &
#         (movies.Oscars >= oscars.value)
#     ]
    
#     if (genre_val != "All"):
#         selected = selected[selected.Genre.str.contains(genre_val)==True]
#     if (director_val != ""):
#         selected = selected[selected.Director.str.contains(director_val)==True]
#     if (cast_val != ""):
#         selected = selected[selected.Cast.str.contains(cast_val)==True]
    return selected


def update():
    df = select_movies()
#     x_name = axis_map[x_axis.value]
#     y_name = axis_map[y_axis.value]
    x_name = x_axis.value
    y_name = y_axis.value

    p.xaxis.axis_label = x_axis.value
    p.yaxis.axis_label = y_axis.value
    p.title.text = "%d movies selected" % len(df)
    source.data = dict(
        x=df[x_name],
        y=df[y_name],
        color=df["color"],
#         title=df["Title"],
#         year=df["Year"],
#         revenue=df["revenue"],
        alpha=df["alpha"],
        stock_name = df.index,
        stock_market_cap = df['שווי שוק'],
        stock_beta = df['בטא'],
    )

# controls = [reviews, boxoffice, genre, min_year, max_year, oscars, director, cast, x_axis, y_axis]
controls = [cast, x_axis, y_axis]
for control in controls:
    control.on_change('value', lambda attr, old, new: update())

sizing_mode = 'fixed'  # 'scale_width' also looks nice with this example

inputs = widgetbox(*controls, sizing_mode=sizing_mode)
l = layout([
#     [desc],
    [inputs, p],
], sizing_mode=sizing_mode)

update()  # initial load of the data

curdoc().add_root(l)

show(p)

In [9]:
# heb_names = (
#     "שווי שוק",
#     "EPS-רווח למניה",
#     "רווח גולמי למכירות",
#     "רווח תפעולי למכירות",
#     "רווח לפני מס למכירות",
#     "רווח נקי למכירות",
#     "תשואה על ההון העצמי",
#     "תשואה על ההשקעה",
#     "תשואה על הנכסים",
#     "דיבידנט למניה",
#     "תשואות דיבידנט",
#     "מרחק מגבוה 52 שבועות %",
#     "% מרחק מנמוך 52 שבועות",
#     "בטא",
#     "סטיית תקן",
#     "% תשואה מתחילת החודש",
#     "% תשואה 3 חודשים אחרונים",
#     "% תשואה מתחילת השנה",
#     "% תשואה 12 חודשים אחרונים",
#     "מכפיל תזרים",
#     "מכפיל הון",
#     "מכפיל מכירות",
#     "הון עצמי למאזן",
#     "יחס שוטף",
#     "מנוף פיננסי",
#     "מכפיל רווח")

# eng_names = (
#     'market_cap',
#     'EPS',
#     'gross_margin',
#     'operating_margin',
#     'pre-tax_profit_margin',
#     'net_profit_margin',
#     'ROE',
#     'ROI',
#     'ROA',
#     'dividend_per_share',
#     'dividend_yield',
#     '%_from_52_week_high',
#     '%_from_52_week_low',
#     'beta',
#     'standard_dev',
#     'return_this_month',
#     'trailing_3_month_return',
#     'return_this_year',
#     'trailing_12_month_return',
#     'price_to_cashflow',
#     'price_to_book',
#     'price_to_revenues',
#     'bv_to_assets',
#     'cur_assets_to_cur_liab',
#     'liab_to_bv',
#     'PE'
# )

# heb_to_eng_names = dict(zip(heb_names, eng_names))